In [1]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

import warnings
warnings.filterwarnings('ignore')

/Users/quansun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# INPUT_DATA
________________________
DX_bl: baseline diagnosis
__________________________
CN_Diag, MCI_Diag, AD_Diag: healthy control, mild cognitive impairment –MCI-, or Alzheimer’s disease

In [2]:
Input_Data = pd.read_csv('TADPOLE_InputData.csv')
Input_Data.head()

,PTID_Key,EXAMDATE,DX_bl,DXCHANGE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,...,EXAMDATE_UPENNBIOMK9_04_19_17,PHASE_UPENNBIOMK9_04_19_17,BATCH_UPENNBIOMK9_04_19_17,KIT_UPENNBIOMK9_04_19_17,STDS_UPENNBIOMK9_04_19_17,RUNDATE_UPENNBIOMK9_04_19_17,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,COMMENT_UPENNBIOMK9_04_19_17
0,400.0,9/8/05,CN,1.0,74.3,Male,16.0,Not Hisp/Latino,White,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,564.0,9/12/05,AD,3.0,81.3,Male,18.0,Not Hisp/Latino,White,Married,...,9/12/05,ADNI1,UPENNBIOMK9,P06-MP02-MP01,P06-MP02-MP01/2,12/14/16,741.5,239.7,22.83,NaN
2,564.0,3/13/06,AD,3.0,81.3,Male,18.0,Not Hisp/Latino,White,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,564.0,9/12/06,AD,3.0,81.3,Male,18.0,Not Hisp/Latino,White,Married,...,9/13/06,ADNI1,UPENNBIOMK9,P06-MP02-MP01,P06-MP02-MP01/2,12/14/16,601.4,251.7,24.18,NaN
4,564.0,9/12/07,AD,3.0,81.3,Male,18.0,Not Hisp/Latino,White,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# normalize the format of DATE
Input_Data['EXAMDATE'] = pd.to_datetime(Input_Data['EXAMDATE'],errors='coerce')
Input_Data.head()

,PTID_Key,EXAMDATE,DX_bl,DXCHANGE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,...,EXAMDATE_UPENNBIOMK9_04_19_17,PHASE_UPENNBIOMK9_04_19_17,BATCH_UPENNBIOMK9_04_19_17,KIT_UPENNBIOMK9_04_19_17,STDS_UPENNBIOMK9_04_19_17,RUNDATE_UPENNBIOMK9_04_19_17,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,COMMENT_UPENNBIOMK9_04_19_17
0,400.0,2005-09-08,CN,1.0,74.3,Male,16.0,Not Hisp/Latino,White,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,564.0,2005-09-12,AD,3.0,81.3,Male,18.0,Not Hisp/Latino,White,Married,...,9/12/05,ADNI1,UPENNBIOMK9,P06-MP02-MP01,P06-MP02-MP01/2,12/14/16,741.5,239.7,22.83,NaN
2,564.0,2006-03-13,AD,3.0,81.3,Male,18.0,Not Hisp/Latino,White,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,564.0,2006-09-12,AD,3.0,81.3,Male,18.0,Not Hisp/Latino,White,Married,...,9/13/06,ADNI1,UPENNBIOMK9,P06-MP02-MP01,P06-MP02-MP01/2,12/14/16,601.4,251.7,24.18,NaN
4,564.0,2007-09-12,AD,3.0,81.3,Male,18.0,Not Hisp/Latino,White,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Extracting features
fixed_features = ['PTID_Key','EXAMDATE']
selected_features = ['CDRSB','ADAS11','RAVLT_immediate','Hippocampus','WholeBrain','Entorhinal','MidTemp','FDG','AV45','APOE4','AGE']
predicted_features = ['DX_bl','DX','DXCHANGE','ADAS13','Ventricles','MMSE']

df_data = Input_Data[fixed_features + selected_features + predicted_features]
df_data.head()

,PTID_Key,EXAMDATE,CDRSB,ADAS11,RAVLT_immediate,Hippocampus,WholeBrain,Entorhinal,MidTemp,FDG,AV45,APOE4,AGE,DX_bl,DX,DXCHANGE,ADAS13,Ventricles,MMSE
0,400.0,2005-09-08,0.0,10.67,44.0,8336.0,1229740.0,4177.0,27936.0,1.36926,NaN,0.0,74.3,CN,NL,1.0,18.67,118233.0,28.0
1,564.0,2005-09-12,4.5,22.00,22.0,5319.0,1129830.0,1791.0,18422.0,1.09079,NaN,1.0,81.3,AD,Dementia,3.0,31.00,84599.0,20.0
2,564.0,2006-03-13,6.0,19.00,19.0,5446.0,1100060.0,2427.0,16972.0,1.06360,NaN,1.0,81.3,AD,Dementia,3.0,30.00,88580.0,24.0
3,564.0,2006-09-12,3.5,24.00,31.0,5157.0,1095640.0,1596.0,17330.0,1.10384,NaN,1.0,81.3,AD,Dementia,3.0,35.00,90099.0,17.0
4,564.0,2007-09-12,8.0,25.67,23.0,5139.0,1088560.0,1175.0,16398.0,1.03871,NaN,1.0,81.3,AD,Dementia,3.0,37.67,97420.0,19.0


In [5]:
# Look at the infomation of data
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8717 entries, 0 to 8716
Data columns (total 19 columns):
PTID_Key           8715 non-null float64
EXAMDATE           8716 non-null datetime64[ns]
CDRSB              6289 non-null float64
ADAS11             6293 non-null float64
RAVLT_immediate    6242 non-null float64
Hippocampus        5360 non-null float64
WholeBrain         6298 non-null float64
Entorhinal         5304 non-null float64
MidTemp            5304 non-null float64
FDG                2932 non-null float64
AV45               1170 non-null float64
APOE4              8704 non-null float64
AGE                8716 non-null float64
DX_bl              8716 non-null object
DX                 6317 non-null object
DXCHANGE           6317 non-null float64
ADAS13             6230 non-null float64
Ventricles         6150 non-null float64
MMSE               6304 non-null float64
dtypes: datetime64[ns](1), float64(16), object(2)
memory usage: 1.3+ MB


In [6]:
# Sorting data
df_data = df_data.sort_values(by=['PTID_Key'])
df_data.head(10)

# Sorting the EXAMDATE because the EXAMDATE of one objective are disorderly
df_data = df_data.groupby(['PTID_Key']).apply(lambda x: x.sort_values(['EXAMDATE'], ascending = True))
df_data.head(10)

PTID_Key   EXAMDATE  CDRSB  ADAS11  RAVLT_immediate  \
PTID_Key                                                             
1.0      3169       1.0 2010-12-10    1.5   14.00             28.0   
         3170       1.0 2011-04-07    NaN     NaN              NaN   
         3171       1.0 2011-09-08    3.0   17.00             26.0   
2.0      1634       2.0 2006-07-21    4.5   18.67             25.0   
         6683       2.0 2007-01-16    5.0   19.33             20.0   
         6684       2.0 2007-07-25    5.5   21.00             20.0   
         6685       2.0 2008-01-07    NaN     NaN              NaN   
3.0      1239       3.0 2006-05-17    0.0    4.00             49.0   
         1240       3.0 2006-11-30    0.0    2.00             48.0   
         1241       3.0 2007-05-23    0.0    5.00             49.0   

               Hippocampus  WholeBrain  Entorhinal  MidTemp      FDG     AV45  \
PTID_Key                                                                        
1.0      3169       6288.0    904429.0      4081.0  14310.0  0.97265  1.67721   
         3170       6314.0    909689.0      3635.0  14899.0      NaN      NaN   
         3171       6345.0    901352.0      3645.0  14891.0      NaN      NaN   
2.0      1634       4951.0    828140.0      2275.0  11760.0  1.02232      NaN   
         6683          NaN    828686.0         NaN      NaN  0.96829      NaN   
         6684          NaN         NaN         NaN      NaN  1.00526      NaN   
         6685          NaN         NaN         NaN      NaN      NaN      NaN   
3.0      1239       8764.0   1179040.0      4350.0  26574.0  1.46245      NaN   
         1240       8809.0   1186300.0      4370.0  26751.0  1.37111      NaN   
         1241       8683.0   1189930.0      4549.0  26277.0  1.45355      NaN   

               APOE4   AGE DX_bl        DX  DXCHANGE  ADAS13  Ventricles  MMSE  
PTID_Key                                                                        
1.0      3169    1.0  81.6  EMCI       MCI       2.0   21.00     30260.0  27.0  
         3170    1.0  81.6  EMCI       NaN       NaN     NaN     30302.0   NaN  
         3171    1.0  81.6  EMCI       MCI       2.0   26.00     31615.0  24.0  
2.0      1634    0.0  76.6    AD  Dementia       3.0   27.67     62890.0  25.0  
         6683    0.0  76.6    AD  Dementia       3.0   30.33     65800.0  24.0  
         6684    0.0  76.6    AD  Dementia       3.0   31.00         NaN  22.0  
         6685    0.0  76.6    AD       NaN       NaN     NaN         NaN   NaN  
3.0      1239    1.0  62.8    CN        NL       1.0    8.00     44493.0  28.0  
         1240    1.0  62.8    CN        NL       1.0    2.00     45655.0  29.0  
         1241    1.0  62.8    CN        NL       1.0    7.00     45509.0  28.0

In [7]:
# Reseting the index to make dataframe clear
df_data = df_data.reset_index(drop=True)
df_data.head(10)

,PTID_Key,EXAMDATE,CDRSB,ADAS11,RAVLT_immediate,Hippocampus,WholeBrain,Entorhinal,MidTemp,FDG,AV45,APOE4,AGE,DX_bl,DX,DXCHANGE,ADAS13,Ventricles,MMSE
0,1.0,2010-12-10,1.5,14.00,28.0,6288.0,904429.0,4081.0,14310.0,0.97265,1.67721,1.0,81.6,EMCI,MCI,2.0,21.00,30260.0,27.0
1,1.0,2011-04-07,NaN,NaN,NaN,6314.0,909689.0,3635.0,14899.0,NaN,NaN,1.0,81.6,EMCI,NaN,NaN,NaN,30302.0,NaN
2,1.0,2011-09-08,3.0,17.00,26.0,6345.0,901352.0,3645.0,14891.0,NaN,NaN,1.0,81.6,EMCI,MCI,2.0,26.00,31615.0,24.0
3,2.0,2006-07-21,4.5,18.67,25.0,4951.0,828140.0,2275.0,11760.0,1.02232,NaN,0.0,76.6,AD,Dementia,3.0,27.67,62890.0,25.0
4,2.0,2007-01-16,5.0,19.33,20.0,NaN,828686.0,NaN,NaN,0.96829,NaN,0.0,76.6,AD,Dementia,3.0,30.33,65800.0,24.0
5,2.0,2007-07-25,5.5,21.00,20.0,NaN,NaN,NaN,NaN,1.00526,NaN,0.0,76.6,AD,Dementia,3.0,31.00,NaN,22.0
6,2.0,2008-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.6,AD,NaN,NaN,NaN,NaN,NaN
7,3.0,2006-05-17,0.0,4.00,49.0,8764.0,1179040.0,4350.0,26574.0,1.46245,NaN,1.0,62.8,CN,NL,1.0,8.00,44493.0,28.0
8,3.0,2006-11-30,0.0,2.00,48.0,8809.0,1186300.0,4370.0,26751.0,1.37111,NaN,1.0,62.8,CN,NL,1.0,2.00,45655.0,29.0
9,3.0,2007-05-23,0.0,5.00,49.0,8683.0,1189930.0,4549.0,26277.0,1.45355,NaN,1.0,62.8,CN,NL,1.0,7.00,45509.0,28.0


In [8]:
# Check how many missing data and the type of data
print(df_data.isnull().sum())
print(df_data.info())

PTID_Key              0
EXAMDATE              0
CDRSB              2427
ADAS11             2423
RAVLT_immediate    2474
Hippocampus        3355
WholeBrain         2418
Entorhinal         3412
MidTemp            3412
FDG                5784
AV45               7546
APOE4                12
AGE                   0
DX_bl                 0
DX                 2399
DXCHANGE           2399
ADAS13             2486
Ventricles         2566
MMSE               2412
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8715 entries, 0 to 8714
Data columns (total 19 columns):
PTID_Key           8715 non-null float64
EXAMDATE           8715 non-null datetime64[ns]
CDRSB              6288 non-null float64
ADAS11             6292 non-null float64
RAVLT_immediate    6241 non-null float64
Hippocampus        5360 non-null float64
WholeBrain         6297 non-null float64
Entorhinal         5303 non-null float64
MidTemp            5303 non-null float64
FDG                2931 non-null float64
AV45   

In [9]:
## Starting to deal with predicted_features
# 1st: look how many different values 'DX_bl','DX','DXCHANGE' have
print(np.unique(df_data['DX_bl'].values))
print('-'*100)
print(np.unique(df_data['DX'].astype(str).values))
print('-'*100)
print(np.unique(df_data['DXCHANGE'].astype(str).values))

['AD' 'CN' 'EMCI' 'LMCI' 'SMC']
----------------------------------------------------------------------------------------------------
['Dementia' 'Dementia to MCI' 'MCI' 'MCI to Dementia' 'MCI to NL' 'NL'
 'NL to Dementia' 'NL to MCI' 'nan']
----------------------------------------------------------------------------------------------------
['1.0' '2.0' '3.0' '4.0' '5.0' '6.0' '7.0' '8.0' 'nan']


--------
Means of different numbers of DXCHANGE is:

1=Stable:NL to NL, 2=Stable:MCI to MCI, 3=Stable:AD to AD, 4=Conv:NL to MCI, 5=Conv:MCI to AD, 6=Conv:NL to AD, 7=Rev:MCI to NL, 8=Rev:AD to MCI

They are similiar with DX. We can find the DX and DXCHANGE have same distribution by observing data. DXCHANGE data is missing when DX data missing, or they have same values. So, we can drop the DXCHANGE.
__________

_______
先对DX进行编码, DXCHANGE把str变成int, DX/DXCHANGE中nan转换成-1

"NL": 1, "MCI": 2, "Dementia": 3, "NL to MCI": 4, "MCI to Dementia": 5, "NL to Dementia":6, "MCI to NL":7,"Dementia to MCI":8
__________

In [10]:
DX_mapping = {"NL": 1, "MCI": 2, "Dementia": 3, "NL to MCI": 4, "MCI to Dementia": 5, "NL to Dementia":6, "MCI to NL":7,"Dementia to MCI":8}
df_data['DX'] = df_data['DX'].map(DX_mapping)
df_data.head(10)

,PTID_Key,EXAMDATE,CDRSB,ADAS11,RAVLT_immediate,Hippocampus,WholeBrain,Entorhinal,MidTemp,FDG,AV45,APOE4,AGE,DX_bl,DX,DXCHANGE,ADAS13,Ventricles,MMSE
0,1.0,2010-12-10,1.5,14.00,28.0,6288.0,904429.0,4081.0,14310.0,0.97265,1.67721,1.0,81.6,EMCI,2.0,2.0,21.00,30260.0,27.0
1,1.0,2011-04-07,NaN,NaN,NaN,6314.0,909689.0,3635.0,14899.0,NaN,NaN,1.0,81.6,EMCI,NaN,NaN,NaN,30302.0,NaN
2,1.0,2011-09-08,3.0,17.00,26.0,6345.0,901352.0,3645.0,14891.0,NaN,NaN,1.0,81.6,EMCI,2.0,2.0,26.00,31615.0,24.0
3,2.0,2006-07-21,4.5,18.67,25.0,4951.0,828140.0,2275.0,11760.0,1.02232,NaN,0.0,76.6,AD,3.0,3.0,27.67,62890.0,25.0
4,2.0,2007-01-16,5.0,19.33,20.0,NaN,828686.0,NaN,NaN,0.96829,NaN,0.0,76.6,AD,3.0,3.0,30.33,65800.0,24.0
5,2.0,2007-07-25,5.5,21.00,20.0,NaN,NaN,NaN,NaN,1.00526,NaN,0.0,76.6,AD,3.0,3.0,31.00,NaN,22.0
6,2.0,2008-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.6,AD,NaN,NaN,NaN,NaN,NaN
7,3.0,2006-05-17,0.0,4.00,49.0,8764.0,1179040.0,4350.0,26574.0,1.46245,NaN,1.0,62.8,CN,1.0,1.0,8.00,44493.0,28.0
8,3.0,2006-11-30,0.0,2.00,48.0,8809.0,1186300.0,4370.0,26751.0,1.37111,NaN,1.0,62.8,CN,1.0,1.0,2.00,45655.0,29.0
9,3.0,2007-05-23,0.0,5.00,49.0,8683.0,1189930.0,4549.0,26277.0,1.45355,NaN,1.0,62.8,CN,1.0,1.0,7.00,45509.0,28.0


In [11]:
# Dropping DXCHANGE
df_data = df_data.drop('DXCHANGE',axis=1)
df_data.head()

,PTID_Key,EXAMDATE,CDRSB,ADAS11,RAVLT_immediate,Hippocampus,WholeBrain,Entorhinal,MidTemp,FDG,AV45,APOE4,AGE,DX_bl,DX,ADAS13,Ventricles,MMSE
0,1.0,2010-12-10,1.5,14.00,28.0,6288.0,904429.0,4081.0,14310.0,0.97265,1.67721,1.0,81.6,EMCI,2.0,21.00,30260.0,27.0
1,1.0,2011-04-07,NaN,NaN,NaN,6314.0,909689.0,3635.0,14899.0,NaN,NaN,1.0,81.6,EMCI,NaN,NaN,30302.0,NaN
2,1.0,2011-09-08,3.0,17.00,26.0,6345.0,901352.0,3645.0,14891.0,NaN,NaN,1.0,81.6,EMCI,2.0,26.00,31615.0,24.0
3,2.0,2006-07-21,4.5,18.67,25.0,4951.0,828140.0,2275.0,11760.0,1.02232,NaN,0.0,76.6,AD,3.0,27.67,62890.0,25.0
4,2.0,2007-01-16,5.0,19.33,20.0,NaN,828686.0,NaN,NaN,0.96829,NaN,0.0,76.6,AD,3.0,30.33,65800.0,24.0


In [12]:
# Filling the missing data in DX
df_data.DX.fillna(method='ffill',inplace=True)
df_data['DX'] = df_data['DX'].astype(int)
df_data.head()

,PTID_Key,EXAMDATE,CDRSB,ADAS11,RAVLT_immediate,Hippocampus,WholeBrain,Entorhinal,MidTemp,FDG,AV45,APOE4,AGE,DX_bl,DX,ADAS13,Ventricles,MMSE
0,1.0,2010-12-10,1.5,14.00,28.0,6288.0,904429.0,4081.0,14310.0,0.97265,1.67721,1.0,81.6,EMCI,2,21.00,30260.0,27.0
1,1.0,2011-04-07,NaN,NaN,NaN,6314.0,909689.0,3635.0,14899.0,NaN,NaN,1.0,81.6,EMCI,2,NaN,30302.0,NaN
2,1.0,2011-09-08,3.0,17.00,26.0,6345.0,901352.0,3645.0,14891.0,NaN,NaN,1.0,81.6,EMCI,2,26.00,31615.0,24.0
3,2.0,2006-07-21,4.5,18.67,25.0,4951.0,828140.0,2275.0,11760.0,1.02232,NaN,0.0,76.6,AD,3,27.67,62890.0,25.0
4,2.0,2007-01-16,5.0,19.33,20.0,NaN,828686.0,NaN,NaN,0.96829,NaN,0.0,76.6,AD,3,30.33,65800.0,24.0


-----------
Coding for DX_bl:

>Alzheimer's Disease (AD) -> 1

>Cognitively Normal (CN) -> 2

>Early Mild Cognitive Impairment (EMCI) -> 3

>Late Mild Cognitive Impairment (LMCI) -> 3

>Significant Memory Concern (SMC) -> 3

>Mild Cognitive Impairment (MCI)
_________________

In [13]:
DXbl_mapping = {"AD": 2, "CN": 0, "EMCI": 1, "LMCI": 1, "SMC": 1}
df_data['DX_bl'] = df_data['DX_bl'].map(DXbl_mapping)

In [14]:
# APOE4 使用mode填充

apoe4_nan = df_data[df_data.APOE4.isnull()]
apoe4_nan_unique = np.unique(apoe4_nan[['DX_bl','DX']].values,axis=0)
for unique in apoe4_nan_unique:
    df_apoe4_nan_mode = df_data[(df_data['DX_bl'] == unique[0]) & (df_data['DX'] == unique[1])].APOE4.mode()[0]
    df_apoe4_nan = apoe4_nan[(apoe4_nan['DX_bl'] == unique[0]) &(apoe4_nan['DX'] == unique[1])] 
    idce = set(df_apoe4_nan.index.values)
    for idx in idce:
        df_data.loc[idx,'APOE4'] = df_apoe4_nan_mode

------
对于出现nan对feature，以DX_bl和DX为基准，计算input_data和出现nan的项中，DX_bl和DX相同值项中input_data的feature的平均值，然后填入nan
_____

In [15]:
# nanList = ['CDRSB','ADAS11','RAVLT_immediate','Hippocampus','WholeBrain','Entorhinal','MidTemp','FDG','AV45','ADAS13','Ventricles','MMSE']
# for nan in nanList:
#     feature_nan = df_data[df_data[nan].isnull()]
#     feature_nan_unique = np.unique(feature_nan[['DX_bl','DX']].values,axis=0)
#     for unique in feature_nan_unique:
#         df_nan_mean = df_data[(df_data['DX_bl'] == unique[0]) & (df_data['DX'] == unique[1])][nan].mean()
#         df_feature_nan = feature_nan[(feature_nan['DX_bl'] == unique[0]) &(feature_nan['DX'] == unique[1])] 
#         idce = df_feature_nan.index.values
#         for idx in idce:
#             df_data.loc[idx,nan] = df_nan_mean

In [16]:
# df_data.isnull().sum()

-----
Still several missing data

That means there is no same status(DX_bl and DX) objective in non-missing data

So we can find a detail, DX can represent the information of DX_bl.
_______

>DX=1 means baseline is NL, and now the status is NL

>DX=2 means baseline is MCI, and now the status is MCI

>DX=3 means baseline is AD, and now the status is AD

>DX=4 means baseline is NL, and now status is MCI

>DX=5 means baseline is MCI, and now status is AD

>DX=6 means baseline is NL, and now status is AD

>DX=7 means baseline is MCI, and now status is NL

>DX=8 means baseline is AD, and now status is MCI

>>DX=1,7 means the status is NL; DX=2,4,8 means the status is MCI; DX=3,5,6 means the status is AD.

In [17]:
nanList = ['CDRSB','ADAS11','RAVLT_immediate','Hippocampus','WholeBrain','Entorhinal','MidTemp','FDG','AV45','ADAS13','Ventricles','MMSE']
for nan in nanList:
    feature_nan = df_data[df_data[nan].isnull()]
    feature_nan_unique = np.unique(feature_nan[['DX']].values)
    for unique in feature_nan_unique:
        df_nan_mean = df_data[(df_data['DX'] == unique)][nan].mean()
        df_feature_nan = feature_nan[(feature_nan['DX'] == unique)] 
        idce = df_feature_nan.index.values
        for idx in idce:
            df_data.loc[idx,nan] = df_nan_mean

In [18]:
df_data.isnull().sum()

PTID_Key           0
EXAMDATE           0
CDRSB              0
ADAS11             0
RAVLT_immediate    0
Hippocampus        3
WholeBrain         0
Entorhinal         3
MidTemp            3
FDG                0
AV45               3
APOE4              0
AGE                0
DX_bl              0
DX                 0
ADAS13             0
Ventricles         0
MMSE               0
dtype: int64

In [19]:
df_data[df_data['Hippocampus'].isnull()]
# we can find the 3 rows miss 'Hippocampus','Enthorhinal' and 'MidTemp' simultaneously
# And they are all transfer from NL into AD
# So we can use the data of AD to fill, that is DX=3,5,6

,PTID_Key,EXAMDATE,CDRSB,ADAS11,RAVLT_immediate,Hippocampus,WholeBrain,Entorhinal,MidTemp,FDG,AV45,APOE4,AGE,DX_bl,DX,ADAS13,Ventricles,MMSE
810,153.0,2009-11-03,4.5,8.670,22.0,NaN,951528.0,NaN,NaN,1.17847,NaN,0.0,83.1,0,6,13.670,39957.0,28.0
811,153.0,2010-05-20,5.0,11.835,32.0,NaN,951528.0,NaN,NaN,1.17118,NaN,0.0,83.1,0,6,16.835,39957.0,27.0
2019,391.0,2009-09-16,5.5,15.000,42.0,NaN,951528.0,NaN,NaN,1.16389,NaN,0.0,80.0,0,6,20.000,39957.0,26.0


In [20]:
df_data[df_data['Hippocampus'].isnull()] = df_data[(df_data['DX']==3) | (df_data['DX']==5)].Hippocampus.mean()

df_data[df_data['Entorhinal'].isnull()] = df_data[(df_data['DX']==3) | (df_data['DX']==5)].Entorhinal.mean()

df_data[df_data['MidTemp'].isnull()] = df_data[(df_data['DX']==3) | (df_data['DX']==5)].MidTemp.mean()

In [21]:
df_data.isnull().sum()
# all missing data have been filled

PTID_Key           0
EXAMDATE           0
CDRSB              0
ADAS11             0
RAVLT_immediate    0
Hippocampus        0
WholeBrain         0
Entorhinal         0
MidTemp            0
FDG                0
AV45               0
APOE4              0
AGE                0
DX_bl              0
DX                 0
ADAS13             0
Ventricles         0
MMSE               0
dtype: int64

------
Now add the delta_time
______

In [22]:
df_data.head(10)

,PTID_Key,EXAMDATE,CDRSB,ADAS11,RAVLT_immediate,Hippocampus,WholeBrain,Entorhinal,MidTemp,FDG,AV45,APOE4,AGE,DX_bl,DX,ADAS13,Ventricles,MMSE
0,1.0,2010-12-10 00:00:00,1.500000,14.000000,28.000000,6288.000000,9.044290e+05,4081.000000,14310.000000,0.972650,1.677210,1.0,81.6,1.0,2.0,21.000000,30260.000000,27.000000
1,1.0,2011-04-07 00:00:00,1.662879,10.447551,32.868298,6314.000000,9.096890e+05,3635.000000,14899.000000,1.221864,1.209350,1.0,81.6,1.0,2.0,16.779408,30302.000000,27.353912
2,1.0,2011-09-08 00:00:00,3.000000,17.000000,26.000000,6345.000000,9.013520e+05,3645.000000,14891.000000,1.221864,1.209350,1.0,81.6,1.0,2.0,26.000000,31615.000000,24.000000
3,2.0,2006-07-21 00:00:00,4.500000,18.670000,25.000000,4951.000000,8.281400e+05,2275.000000,11760.000000,1.022320,1.381244,0.0,76.6,2.0,3.0,27.670000,62890.000000,25.000000
4,2.0,2007-01-16 00:00:00,5.000000,19.330000,20.000000,5510.561966,8.286860e+05,2703.417035,16480.752212,0.968290,1.381244,0.0,76.6,2.0,3.0,30.330000,65800.000000,24.000000
5,2.0,2007-07-25 00:00:00,5.500000,21.000000,20.000000,5510.561966,9.546711e+05,2703.417035,16480.752212,1.005260,1.381244,0.0,76.6,2.0,3.0,31.000000,53738.676316,22.000000
6,2.0,2008-01-07 00:00:00,5.761086,22.384435,20.421344,5510.561966,9.546711e+05,2703.417035,16480.752212,1.046708,1.381244,0.0,76.6,2.0,3.0,32.872484,53738.676316,21.484291
7,3.0,2006-05-17 00:00:00,0.000000,4.000000,49.000000,8764.000000,1.179040e+06,4350.000000,26574.000000,1.462450,1.108414,1.0,62.8,0.0,1.0,8.000000,44493.000000,28.000000
8,3.0,2006-11-30 00:00:00,0.000000,2.000000,48.000000,8809.000000,1.186300e+06,4370.000000,26751.000000,1.371110,1.108414,1.0,62.8,0.0,1.0,2.000000,45655.000000,29.000000
9,3.0,2007-05-23 00:00:00,0.000000,5.000000,49.000000,8683.000000,1.189930e+06,4549.000000,26277.000000,1.453550,1.108414,1.0,62.8,0.0,1.0,7.000000,45509.000000,28.000000


In [23]:
id_list = np.unique(df_data['PTID_Key'].values)
for ID in id_list:
    timeList = df_data[df_data['PTID_Key']==ID].EXAMDATE.values
    t0 = df_data[df_data['PTID_Key']==ID].EXAMDATE.values[0]
    for n in range(len(timeList)):
        idx = df_data[df_data['PTID_Key']==ID].index[n]
        df_data.loc[idx,'deltaTime'] = pd.Timedelta(timeList[n] - t0).days

In [24]:
# drop the noise data
df_data.drop(index = df_data[df_data['DX']==5537].index,inplace=True)

In [25]:
df_data['DX_bl'] = df_data['DX_bl'].astype(int)

------
transfer the DX in df_input into the status of objective

>DX=1,7 means the status is NL; DX=2,4,8 means the status is MCI; DX=3,5,6 means the status is AD.
________

In [26]:
index_value1 = df_data[(df_data['DX']==1) | (df_data['DX']==7)].index.values
for idx in index_value1:
    df_data.loc[idx,'DX'] = 0

index_value2 = df_data[(df_data['DX']==2) | (df_data['DX']==4) | (df_data['DX']==8)].index.values
for idx in index_value2:
    df_data.loc[idx,'DX'] = 1

index_value3 = df_data[(df_data['DX']==3) | (df_data['DX']==5) | (df_data['DX']==6)].index.values
for idx in index_value3:
    df_data.loc[idx,'DX'] = 2
    
# transform the type of 'DX'
df_data['DX'] = df_data['DX'].astype(int)

## Train_target

In [27]:
train_target = pd.read_csv('TADPOLE_TargetData_train.csv')
train_target.head()

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE
0,7/10/13,785,0.0,1.0,0.0,5.0,0.012737,28.0
1,1/15/13,785,NaN,NaN,NaN,NaN,NaN,NaN
2,1/17/14,785,NaN,NaN,NaN,NaN,NaN,NaN
3,7/29/14,785,1.0,0.0,0.0,7.0,NaN,30.0
4,7/24/15,785,1.0,0.0,0.0,11.0,0.013934,25.0


In [28]:
# normalize the format of DATE
train_target['Date'] = pd.to_datetime(train_target['Date'])

# Sorting data
train_target = train_target.sort_values(by=['PTID_Key'])

# Sorting the Date because the Date of one objective are disorderly
train_target = train_target.groupby(['PTID_Key']).apply(lambda x: x.sort_values(['Date'], ascending = True))

train_target = train_target.reset_index(drop=True)

In [29]:
train_target.head(20)

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE
0,2013-06-07,6,1.0,0.0,0.0,NaN,NaN,30.0
1,2013-10-16,6,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-05-29,6,1.0,0.0,0.0,3.0,NaN,30.0
3,2016-05-05,6,1.0,0.0,0.0,1.0,NaN,29.0
4,2013-03-28,8,NaN,NaN,NaN,NaN,NaN,NaN
5,2013-10-31,8,0.0,1.0,0.0,5.0,0.012128,30.0
6,2014-04-28,8,NaN,NaN,NaN,NaN,NaN,NaN
7,2013-02-04,18,0.0,1.0,0.0,9.0,0.020526,30.0
8,2013-09-03,18,NaN,NaN,NaN,NaN,NaN,NaN
9,2014-01-29,18,0.0,1.0,0.0,12.0,NaN,29.0


-----------
train_target中cn_diag,mci_diag,ad_diag缺失时adas13和mmse也缺失
______________

In [30]:
# Check missing data
print(train_target.isnull().sum())
print("-"*100)
print(train_target.info())

Date                  0
PTID_Key              0
CN_Diag             877
MCI_Diag            877
AD_Diag             877
ADAS13              869
Ventricles_Norm    1638
MMSE                848
dtype: int64
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2506 entries, 0 to 2505
Data columns (total 8 columns):
Date               2506 non-null datetime64[ns]
PTID_Key           2506 non-null int64
CN_Diag            1629 non-null float64
MCI_Diag           1629 non-null float64
AD_Diag            1629 non-null float64
ADAS13             1637 non-null float64
Ventricles_Norm    868 non-null float64
MMSE               1658 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 156.7 KB
None


-----
Filling the missing data of train_target

Calculating the mean of same status(same values of cn_diag, mci_diag and ad_diag

But first we should fill the missing data about status
_________

In [31]:
idList = np.unique(train_target['PTID_Key'].values)
for ID in idList:
    first_value = train_target[train_target['PTID_Key']==ID]['CN_Diag'].values[0]
    first_idx = train_target[train_target['PTID_Key']==ID].index[0]
    if first_value== 'nan':
        train_target.drop(index=first_value,inplace=True) 

In [32]:
train_target.CN_Diag.fillna(method='ffill',inplace=True)
train_target.MCI_Diag.fillna(method='ffill',inplace=True)
train_target.AD_Diag.fillna(method='ffill',inplace=True)

-----
Filling the missing data of other columns
_____

In [33]:
# 0 represents CN, 1 represents MCI, 2 represents AD
dig_list = ['CN_Diag','MCI_Diag','AD_Diag']

dig_target = np.argmax(train_target[dig_list].values,axis=1)

train_target['DX'] = dig_target

train_target.head()

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE,DX
0,2013-06-07,6,1.0,0.0,0.0,NaN,NaN,30.0,0
1,2013-10-16,6,1.0,0.0,0.0,NaN,NaN,NaN,0
2,2014-05-29,6,1.0,0.0,0.0,3.0,NaN,30.0,0
3,2016-05-05,6,1.0,0.0,0.0,1.0,NaN,29.0,0
4,2013-03-28,8,1.0,0.0,0.0,NaN,NaN,NaN,0


In [34]:
train_columns = ['ADAS13','Ventricles_Norm','MMSE']
for column in train_columns:
    feature_nan = train_target[train_target[column].isnull()]
    feature_nan_unique = np.unique(feature_nan['DX'].values)
    for unique in feature_nan_unique:
        df_nan_mean = train_target[(train_target['DX'] == unique)][column].mean()
        df_feature_nan = feature_nan[(feature_nan['DX'] == unique)] 
        idce = df_feature_nan.index.values
        for idx in idce:
            train_target.loc[idx,column] = df_nan_mean

In [35]:
train_target['CN_Diag'] = train_target['CN_Diag'].astype(int)
train_target['MCI_Diag'] = train_target['MCI_Diag'].astype(int)
train_target['AD_Diag'] = train_target['AD_Diag'].astype(int)

In [36]:
train_target.isnull().sum()

Date               0
PTID_Key           0
CN_Diag            0
MCI_Diag           0
AD_Diag            0
ADAS13             0
Ventricles_Norm    0
MMSE               0
DX                 0
dtype: int64

# Validation_target

In [37]:
validation_target = pd.read_csv('TADPOLE_TargetData_validation.csv')

# normalize the format of DATE
validation_target['Date'] = pd.to_datetime(validation_target['Date'])

# Sorting data
validation_target = validation_target.sort_values(by=['PTID_Key'])

# Sorting the Date because the Date of one objective are disorderly
validation_target = validation_target.groupby(['PTID_Key']).apply(lambda x: x.sort_values(['Date'], ascending = True))

validation_target = validation_target.reset_index(drop=True)

idList_val = np.unique(validation_target['PTID_Key'].values)
for ID in idList_val:
    first_value = validation_target[validation_target['PTID_Key']==ID]['CN_Diag'].values[0]
    first_idx = validation_target[validation_target['PTID_Key']==ID].index[0]
    if first_value== 'nan':
        validation_target.drop(index=first_value,inplace=True) 
validation_target.CN_Diag.fillna(method='ffill',inplace=True)
validation_target.MCI_Diag.fillna(method='ffill',inplace=True)
validation_target.AD_Diag.fillna(method='ffill',inplace=True)

# 0 represents CN, 1 represents MCI, 2 represents AD
dig_list = ['CN_Diag','MCI_Diag','AD_Diag']

dig_target_val = np.argmax(validation_target[dig_list].values,axis=1)

validation_target['DX'] = dig_target_val

val_columns = ['ADAS13','Ventricles_Norm','MMSE']
for column in val_columns:
    feature_nan = validation_target[validation_target[column].isnull()]
    feature_nan_unique = np.unique(feature_nan['DX'].values)
    for unique in feature_nan_unique:
        df_nan_mean = validation_target[(validation_target['DX'] == unique)][column].mean()
        df_feature_nan = feature_nan[(feature_nan['DX'] == unique)] 
        idce = df_feature_nan.index.values
        for idx in idce:
            validation_target.loc[idx,column] = df_nan_mean
            

validation_target['CN_Diag'] = validation_target['CN_Diag'].astype(int)
validation_target['CN_Diag'] = validation_target['CN_Diag'].astype(int)
validation_target['CN_Diag'] = validation_target['CN_Diag'].astype(int)

# Test_target

In [38]:
test_target = pd.read_csv('TADPOLE_PredictTargetData_test.csv')
test_target.head()

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE
0,1/25/13,583,NaN,NaN,NaN,NaN,NaN,NaN
1,6/25/13,583,NaN,NaN,NaN,NaN,NaN,NaN
2,3/10/14,583,NaN,NaN,NaN,NaN,NaN,NaN
3,12/4/13,809,NaN,NaN,NaN,NaN,NaN,NaN
4,4/25/13,809,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
test_target = pd.read_csv('TADPOLE_PredictTargetData_test.csv')

# normalize the format of DATE
test_target['Date'] = pd.to_datetime(test_target['Date'])

# Sorting data
test_target = test_target.sort_values(by=['PTID_Key'])

# Sorting the Date because the Date of one objective are disorderly
test_target = test_target.groupby(['PTID_Key']).apply(lambda x: x.sort_values(['Date'], ascending = True))

test_target = test_target.reset_index(drop=True)

In [40]:
test_target.head()

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE
0,2013-04-15,4,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-10-15,4,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-04-16,4,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-06-05,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-12-11,10,NaN,NaN,NaN,NaN,NaN,NaN


-------
Calculating the delta_time of train_target and validation target
_______

In [41]:
ID_train = np.unique(train_target.PTID_Key.values)
for ID in ID_train:
    idx_list = train_target[train_target['PTID_Key']==ID].index.values
    for idx in idx_list:
        train_target.loc[idx,'blTime'] = df_data[df_data['PTID_Key']==ID]['EXAMDATE'].values[0]
        train_target.loc[idx,'DX_bl'] = df_data[df_data['PTID_Key']==ID]['DX'].values[0]
train_target['DX_bl'] = train_target['DX_bl'].astype(int)
        
ID_validation = np.unique(validation_target.PTID_Key.values)
for ID in ID_validation:
    idx_list = validation_target[validation_target['PTID_Key']==ID].index.values
    for idx in idx_list:
        validation_target.loc[idx,'blTime'] = df_data[df_data['PTID_Key']==ID]['EXAMDATE'].values[0]
        validation_target.loc[idx,'DX_bl'] = df_data[df_data['PTID_Key']==ID]['DX'].values[0]
validation_target['DX_bl'] = validation_target['DX_bl'].astype(int)
        
ID_test = np.unique(test_target.PTID_Key.values)
for ID in ID_test:
    idx_list = test_target[test_target['PTID_Key']==ID].index.values
    for idx in idx_list:
        test_target.loc[idx,'blTime'] = df_data[df_data['PTID_Key']==ID]['EXAMDATE'].values[0]
        test_target.loc[idx,'DX_bl'] = df_data[df_data['PTID_Key']==ID]['DX'].values[0]
test_target['DX_bl'] = test_target['DX_bl'].astype(int)

In [42]:
train_target.head()

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE,DX,blTime,DX_bl
0,2013-06-07,6,1,0,0,8.055944,0.022799,30.000000,0,2006-04-26,0
1,2013-10-16,6,1,0,0,8.055944,0.022799,28.955326,0,2006-04-26,0
2,2014-05-29,6,1,0,0,3.000000,0.022799,30.000000,0,2006-04-26,0
3,2016-05-05,6,1,0,0,1.000000,0.022799,29.000000,0,2006-04-26,0
4,2013-03-28,8,1,0,0,8.055944,0.022799,28.955326,0,2011-10-27,0


In [43]:
for idx in train_target.index.values:
    train_target.loc[idx,'deltaTime'] = pd.Timedelta(train_target.loc[idx,'Date'] -  train_target.loc[idx,'blTime']).days
    
for idx in validation_target.index.values:
    validation_target.loc[idx,'deltaTime'] = pd.Timedelta(validation_target.loc[idx,'Date'] - validation_target.loc[idx,'blTime']).days
    
for idx in test_target.index.values:
    test_target.loc[idx,'deltaTime'] = pd.Timedelta(test_target.loc[idx,'Date'] - test_target.loc[idx,'blTime']).days
    

In [44]:
train_target.drop(['Date','blTime'],axis=1,inplace=True)
validation_target.drop(['Date','blTime'],axis=1,inplace=True)

In [45]:
df_data['Ventricles_Norm'] = df_data['Ventricles']
df_data.drop(['Ventricles'],axis=1,inplace=True)

In [46]:
train_target.head()

,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE,DX,DX_bl,deltaTime
0,6,1,0,0,8.055944,0.022799,30.000000,0,0,2599.0
1,6,1,0,0,8.055944,0.022799,28.955326,0,0,2730.0
2,6,1,0,0,3.000000,0.022799,30.000000,0,0,2955.0
3,6,1,0,0,1.000000,0.022799,29.000000,0,0,3662.0
4,8,1,0,0,8.055944,0.022799,28.955326,0,0,518.0


In [47]:
# train_target.to_csv('./train_target.csv',index=False)
# validation_target.to_csv('./validation_target.csv',index=False)

# OUTPUT CSV

In [48]:
# drop the noise data
df_data.drop(index = df_data[df_data['DX']==5537].index,inplace=True)

In [49]:
df_data.to_csv('./df_data.csv',index=False)
train_target.to_csv('./train_target.csv',index=False)
validation_target.to_csv('./validation_target.csv',index=False)
test_target.to_csv('./test_target.csv',index=False)

In [50]:
data_1st = df_data.drop(['PTID_Key','EXAMDATE'],axis=1)
data_1st.to_csv('./data_1st.csv',index=False)

train_1st = train_target.drop(['CN_Diag','MCI_Diag','AD_Diag'],axis=1)
train_1st.to_csv('./train_1st.csv',index=False)

validation_1st = validation_target.drop(['CN_Diag','MCI_Diag','AD_Diag'],axis=1)
validation_1st.to_csv('./validation_1st.csv',index=False)

In [51]:
data_delta = data_1st[['DX_bl','DX','ADAS13','Ventricles_Norm','MMSE','deltaTime']]
data_delta.to_csv('./data_delta.csv',index=False)